In [201]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

class BabyNames:
    
    def __init__(self):
        self.folder = None
        self.data = None
    
    def Constructor(self, file_location = None):
        
        # Create list of state abbreviations
#         states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
#                   "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
#                   "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
#                   "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
#                   "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
        
        # Testing states
        states = ["AL", "AK", "AZ", "AR", "CA", "CO"]
        
        # Init df
        df = pd.DataFrame()
        
        # Read all state babynames files into df
        for i in states:
            temp = pd.read_csv(file_location + "/" + str(i) + ".TXT", sep=",", header = None)
            df = pd.concat([df, temp])
        
        # Rename columns
        df = df.rename(columns={0: "state", 1: "sex", 2: "year", 3: "name", 4: "births"})
        
        # Store df in class
        self.data = df
        self.folder = file_location
        return self
    
    
    def Count(self, state = None, year = None):
        
        # Load class data
        df = self.data
        
        # Return sum births from df
        if state and year: return sum(df[(df.state == state) & (df.year == year)].births)

        elif not state and year: return sum(df[(df.year == year)].births)
            
        elif state and not year: return sum(df[(df.state == state)].births)
            
        else: return sum(df.births)
    
    def Top10BabyNames(self, state = "IL", year = 2015):
        
        # Load class data
        df = self.data
        
        # Get top ten names and ranks
        df_m = df[(df.state == state) & 
                  (df.year  == year) & 
                  (df.sex   == "M")].sort_values(by = 'births', ascending = False).head(10)
        
        df_f = df[(df.state == state) & 
                  (df.year  == year) & 
                  (df.sex   == "F")].sort_values(by = 'births', ascending = False).head(10).name
        
        df_m["ranks"]  = list(range(1, 11))
        df_m["name_f"] = list(df_f)
        
        # Return ranked table
        return df_m[["ranks", "name", "name_f"]]
    
    def ChangeOfPopularity(self, fromYear = 2014, toYear = 2015, top = 10):
        
        df = self.data
        
        # Get compare dataframe
        df = df[(df.year == fromYear) | (df.year == toYear)].groupby(["year", 
                                                                      "name"]).agg({"births": "sum"})
        df = df.reset_index()
        df_initial = df[df.year == fromYear]
        df_final   = df[df.year == toYear]
        
        df = pd.merge(df_initial, df_final, on = "name")
        df["change"] = df["births_y"] - df["births_x"] 
        
        # Get names that increased in popularity
        df_increased = df.sort_values(by = ["change"], ascending = False).head(top)
        
        # Get names that decreased in popularity
        df_decreased = df.sort_values(by = ["change"], ascending = True).head(top)
        dec_names, dec_by = list(df_decreased.name), list(df_decreased.change)
        
        # Get names that stayed the same in populatiry
        df_same = df.sort_values(by = ["change"], key = abs).head(top)
        same_names, same_by = list(df_same.name), list(df_same.change)
        
        # Create output table
        df_increased["ranks"]  = list(range(1, top+1))
        
        df_increased["dec_names"]  = dec_names
        df_increased["dec_by"]     = dec_by
        df_increased["same_names"] = same_names
        df_increased["same_by"]    = same_by
        
        df = df_increased[["ranks", "name", "change", 
                             "dec_names", "dec_by", 
                             "same_names", "same_by"]]
        
        df = df.rename(columns={"ranks": "Ranks", "name": "Inc. Names", 
                                "change": "Inc. Change", "dec_names": "Dec. Names", 
                                "dec_by": "Dec. Change", "same_names": "Same Names", 
                                "same_by": "Same Change"})
        
        return df
    
    def Top5NamesPerYear(self, year = 2015, sex = ''):
        # sex argument can only take 'F','M', or ''

        #filtering dataframe based on the given input
        if sex != '':
            df = self.data[(self.data['year'] == year) & (self.data['sex'] == sex)]
            df = df.groupby('state').head(5).reset_index(drop=True)
        else:
            df = self.data[(self.data['year'] == year)]
            g = df.groupby('state')
            df = g.apply(lambda x: x.sort_values(by = 'births', ascending=False).head(5))
        uniq_state = df['state'].unique()
        
        # number of rows the filtered dataframe
        num_rows = df.shape[0]
        ranks = [[],[],[],[],[]]
        nums = [[],[],[],[],[]]
        for i in range(num_rows):
            index = i%5
            ranks[index].append(df.iloc[i,3])
            nums[index].append(df.iloc[i,4])
            
        #construct an output dataframe
        if sex == 'F':
            psex = 'Female'
        elif sex == 'M':
            psex = 'Male'
        elif sex == '':
            psex = 'Female or Male'
        print("Top Five " + psex + " Names for Births in " + str(year))
        output_df = pd.DataFrame({'State': uniq_state,'Rank1': ranks[0],'Num1': nums[0], 'Rank2':ranks[1],
                            'Num2': nums[1], 'Rank3':ranks[2],'Num3': nums[2], 'Rank4':ranks[3],'Num4': nums[3],
                             'Rank5':ranks[4],'Num5': nums[4]})
        return output_df
        
    
    def NamePopularityPlot(self, name = "Jim", yearRange = (2000, 2015), state = "IL", sex = "M"):
        startY = yearRange[0]
        endY = yearRange[1]
        year_list = list(range(yearRange[0],yearRange[1]+1))
        filtered_df = self.data[(self.data['state'] == state) & (self.data['sex'] == sex) &(self.data['name'] == name)]
        filtered_df = filtered_df[(filtered_df['year']>= startY) & (filtered_df['year'] <=  endY)]
        print(filtered_df)
        plt.plot(year_list, filtered_df.births, color='red', marker='o')
        plt.title('Name = '+name+' popularity from '+ str(startY) + " to " + str(endY), fontsize=14)
        plt.xlabel('Year', fontsize=14)
        plt.ylabel('number of births', fontsize=14)
        plt.grid(True)
        plt.show()
        return None
        return self
    
    def NameFlip(self, n = 10):
         # This function returns the top N names that has been flipped the most. 

        # group data by names and year, and shows the corresponding number of births for female and male
        pivoted_df = self.data.pivot_table(values = 'births', index = ['name','year'],columns = 'sex', aggfunc=np.sum)
        pivoted_df = pivoted_df.fillna(0)

        # 'F-M' column calculates the difference between number of females and males using the same name
        # a positive value of F-M indicates a girl name, a negative value of F-M indicates a boy name
        pivoted_df['F-M'] = pivoted_df['F'] - pivoted_df['M']

        # check if the name has been flipped, i.e, the sign of (F-M) is changed. 
        flip_df2 =  pd.DataFrame(pivoted_df.groupby('name')['F-M'].apply(lambda grp: max(grp)*min(grp)))
        flip_df2.columns = ['flip_or_not']
        # only filtering rows with changed signs in F-M columns
        flip_df2 = flip_df2[flip_df2['flip_or_not']<0]
        names_flipped = list(flip_df2.index)
        
        #calculte the number of flips, i.e. max('F-M')-min('F-M') for each name across time. 
        filter_flip = pivoted_df[pivoted_df.index.isin(names_flipped, level=0)]
        filter_flip = pd.DataFrame(filter_flip.groupby('name')['F-M'].apply(lambda grp: max(grp)-min(grp)))
        filter_flip.columns = ['max_flips']
        # sort names based on the 'Max_flips'
        filter_flip = filter_flip.sort_values(by = ['max_flips','name'], ascending = [False,True])
        
        #top n names filpped the most.
        names= list(filter_flip.index[0:n])
        # preparing a dataframe to be plotted latter
        plotted_df = pivoted_df[pivoted_df.index.isin(names, level=0)]
        plotted_df = pd.pivot_table(plotted_df, index = 'year', columns = 'name', values = 'F-M')
        #plotting a graph
        line_plot = plotted_df.plot(figsize=(12,10),grid=False,title='difference between number of females and male (F-M) for each top flipped names across time')
        line_plot.axhline (y=0 , color = 'black')

        # return a list of top n names filpped the most.
        return names

In [202]:
baby_test = BabyNames().Constructor(file_location = r"C:\Users\nuke2\Desktop\NW Work\Data\babynames")

In [203]:
baby_test.ChangeOfPopularity()

,Ranks,Inc. Names,Inc. Change,Dec. Names,Dec. Change,Same Names,Same Change
264,1,Alexa,690,Isabella,-446,Ellieana,0
3851,2,Matteo,491,Anthony,-403,Maximino,0
3839,3,Mateo,428,Andrew,-379,Ammar,0
3954,4,Mia,401,Jayden,-367,Jannet,0
4264,5,Oliver,322,Kevin,-355,Bibiana,0
2284,6,Hazel,256,Joshua,-306,Siddhartha,0
1266,7,Charlotte,254,Sophia,-298,Maxon,0
140,8,Adriel,245,Alexander,-273,Sidharth,0
4539,9,Riley,224,Annabelle,-224,Letty,0
1970,10,Ezra,221,Damian,-222,Bladimir,0
